# 13.基础Actor-Critic算法
> **Actor-Critic（执行者-评论者）** 算法结合了 **基于值** 的学习方法（如Q学习）和 **基于策略** 的学习方法（如策略梯度）。这种方法将智能体的学习过程分为两个部分： **Actor（执行者）** 和 **Critic（评论者）**，每个部分负责不同的任务：**Actor** 通过策略采取行动，**Critic** 通过价值函数给 **Actor** 反馈当前状态或状态-动作对的“好坏”。其本质上是 **基于策略** 的算法，算法的目标是优化一个带参数的策略，只是会额外学习价值函数，从而帮助策略函数更好地学习。
> **Actor-Critic** 是囊括一系列算法的**整体架构**，目前很多高效的前沿算法都属于 **Actor-Critic** 算法。
> **Actor-Critic** 算法最早的理论基础和框架由 Bertsekas 和 Tsitsiklis 在他们的经典书籍《Neuro-Dynamic Programming》（1989年）中提出，正式提出在[Actor-Critic Algorithms（2000年）](https://proceedings.neurips.cc/paper_files/paper/1999/file/6449f44a102fde848669bdd9eb6b76fa-Paper.pdf)论文中。

> 本节会学习一种最基础的 **Actor-Critic** 算法：